In [1]:
import pandas as pd
#data_df=pd.read_csv("train.csv",nrows=100000000)
data_df=pd.read_csv("train.csv")

In [ ]:
import numpy as np

from datetime import datetime
def feature1(data, test=False):
    if test == 1:
        dealed_data = data.drop_duplicates(['file_id'])['file_id']
    else:
        dealed_data = data.drop_duplicates(['file_id'])[['file_id','label']]
    # 提取各类别特征的类别数
    dealed_train = data.groupby("file_id")[["api",'tid','return_value']].nunique()
    dealed_train.columns = ['api_nunique','tid_nunique','value_nunique']
    
    temp = data.groupby(['file_id'])['index'].count().rename('id_count')
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    # 对每个file_id,的每个feat计数，然后求其统计特征
    cate_feat = ['api','tid','return_value']
    for feat in cate_feat:
        temp = data.groupby(['file_id',feat])[feat].count().groupby(['file_id']).agg(['min','max','mean','median','std',pd.Series.mad,
                                                                            pd.Series.skew,pd.Series.kurt]).add_prefix(feat+'_cnt_')
        dealed_train = pd.concat([dealed_train, temp],axis=1)

    # 提取交叉特征，每个file_id，每个不同api的tid种类数的统计特征
    temp = data.groupby(['file_id','api'])['tid'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('api_tid_')
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    # 提取交叉特征，每个file_id，每个不同api的return种类数的统计特征
    temp = data.groupby(['file_id','api'])['return_value'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('api_value_')                                                               
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    temp = data.groupby(['file_id','tid'])['api'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('tid_tid_')
    dealed_train = pd.concat([dealed_train, temp],axis=1)

    temp = data.groupby(['file_id','tid'])['return_value'].nunique().groupby('file_id').agg(['min','max','mean','median','std',pd.Series.mad,
                                                                    pd.Series.skew,pd.Series.kurt]).add_prefix('tid_value_')                                                               
    dealed_train = pd.concat([dealed_train, temp],axis=1)
    return dealed_train

train_data = feature1(data_df, test=False)
#test_data = feature1(test, test=True)
train_data.to_csv('train_featur_1mil.csv', index=False)
#test_data.to_csv('test_feature1.csv', index=False)

In [ ]:
label=data_df.groupby(['file_id','label'])['label'].unique()


In [ ]:
label.shape

In [ ]:
import numpy as np
train_X=np.array(train_data)
train_y=label


In [ ]:
train_X.shape,train_y.shape

In [ ]:
def runXGB(train_X,train_y,test_X,test_y=None,feature_names=None,seed_val=0,num_rounds=1000):
    #参数设定
    param = {}
    param['objective'] = 'multi:softprob'#多分类、输出概率值
    param['eta'] = 0.1#学习率
    param['max_depth'] = 6#最大深度，越大越容易过拟合
    param['silent'] = 1#打印提示信息
    param['num_class'] = 6#三个类别
    param['eval_metric']= "mlogloss"#对数损失
    param['min_child_weight']=1#停止条件，这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
    param['subsample'] =0.7#随机采样训练样本
    param['colsample_bytree'] = 0.7# 生成树时进行的列采样
    param['seed'] = seed_val#随机数种子
    num_rounds = num_rounds#迭代次数
    
    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X,label=train_y)
    
    if test_y is not None:
        xgtest = xgb.DMatrix(test_X,label=test_y)
        watchlist = [(xgtrain,'train'),(xgtest,'test')]
        model = xgb.train(plst,xgtrain,num_rounds,watchlist,early_stopping_rounds=20)
      #  early_stopping_rounds 当设置的迭代次数较大时，early_stopping_rounds 可在一定的迭代次数内准确率没有提升就停止训练
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst,xgtrain,num_rounds)
    pred_test_y = model.predict(xgtest)
    return pred_test_y,model

In [ ]:
import xgboost as xgb
from sklearn import model_selection,preprocessing,ensemble
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
cv_scores = []
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=0)
pred_test_y,model=runXGB(x_train,y_train,x_valid,y_valid)